In [6]:
import pickle
import glob
import os

In [12]:
# Get all .txt files in the lemmatized_sents directory
file_paths = glob.glob('../data/lemmatized_sents/*.txt')

docs_lemmata = []
for file_path in file_paths:
    doc_lemmata = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
        # Now 'lines' is a list where each element is a line from the file.
        for line in lines:
            # Process each line (each lemmatized sentence) here
            doc_lemmata.extend(line.strip().split())
    docs_lemmata.append(doc_lemmata)

In [13]:
docs_lemmata[0]

['morieni',
 'romanus',
 'qvondam',
 'eremitae',
 'hierosolymitanus',
 'transfiguratio',
 'metallum',
 'occulo',
 'summus',
 'antiquus',
 'philosophus',
 'medicinus',
 'libellus',
 'lux',
 'edo',
 'privilegium',
 'parisiis',
 'gulielmus',
 'guillard',
 'uia',
 'iacobaea',
 'diui',
 'barbara',
 'signum',
 'lex',
 'regia',
 'cautum',
 'gulielmus',
 'guillard',
 'imprimo',
 'libellus',
 'morienus',
 'romanus',
 'transfiguratio',
 'metallorum',
 'impresdo',
 'diuendo',
 'finis',
 'regnum',
 'decennius',
 'candidus',
 'lectori',
 'salutem',
 'oculus',
 'tuus',
 'offero',
 'auriferus',
 'ars',
 'libellus',
 'paucus',
 'uideo',
 'multus',
 'optatus',
 'scriptores',
 'inferus',
 'aetas',
 'crebrò',
 'laudo',
 'locus',
 'proferrem',
 'breuitas',
 'studium',
 'ueto',
 'arabicò',
 'sermo',
 'latinus',
 'conuerto',
 'peruetustus',
 'exemplarium',
 'nacti',
 'alter',
 'interperno',
 'nomen',
 'praefero',
 'alter',
 'supprimo',
 'loquendi',
 'modus',
 'translator',
 'disssideo',
 'collatio',
 'depre